In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [2]:
df = pd.read_csv('../../data/uspto/train.csv', index_col=0)
df.head()

,rxn_smiles,label,reactants,products,group
0,Br.C=C(CCCN1C(=O)c2ccccc2C1=O)C(=O)O>>C=C(CCCN...,0,Br.C=C(CCCN1C(=O)c2ccccc2C1=O)C(=O)O,C=C(CCCNC=O)C(=O)O,0
1,Br.C=C(CCCN1C(=O)c2ccccc2C1=O)C(=O)O>>C=C(CCCN...,0,Br.C=C(CCCN1C(=O)c2ccccc2C1=O)C(=O)O,C=C(CCCN1Cc2ccccc2C1=O)C(=O)O,0
2,Br.CCOC(=O)C(C)(C)CCC=Cc1ccccc1Cl>>CC(C)(CCC=C...,0,Br.CCOC(=O)C(C)(C)CCC=Cc1ccccc1Cl,CC(C)(CCC=Cc1ccccc1Cl)C(=O)O,1
3,Br.CCOC(=O)C(C)(C)CCC=Cc1ccccc1Cl>>CC(C)(CO)CC...,0,Br.CCOC(=O)C(C)(C)CCC=Cc1ccccc1Cl,CC(C)(CO)CCC=Cc1ccccc1Cl,1
4,BrC(Br)(Br)Br.CC(=O)OCc1ccc(Cl)cc1N([C@H](C)CC...,0,BrC(Br)(Br)Br.CC(=O)OCc1ccc(Cl)cc1N([C@H](C)CC...,CC=O,4


In [3]:
from modify_rrd import RRDCalculator
RRDC = RRDCalculator(scale=True)


2022-05-21 20:45:03.786451: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-21 20:45:05.237689: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-21 20:45:05.237728: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: S03-SIAI
2022-05-21 20:45:05.237734: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: S03-SIAI
2022-05-21 20:45:05.237828: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2022-05-21 20:45:05.237859: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported vers

In [4]:
reactant_smiles = df['reactants'][0]
product_smiles = df['products'][0]
reactants = reactant_smiles.split('.')
products = product_smiles.split('.')
reactants_fea = RRDC.batch_transform(reactants, n_jobs=10)
products_fea = RRDC.batch_transform(reactants, n_jobs=10)

reactants_mol = [Chem.MolFromSmiles(i) for i in reactants]
products_mol = [Chem.MolFromSmiles(i) for i in products]

            ###!!!! add H 
reactants_mol = [Chem.AddHs(i) for i in reactants_mol]
products_mol = [Chem.AddHs(i) for i in products_mol]

100%|##########| 2/2 [00:00<00:00, 94.13it/s]
2022-05-21 20:45:14.706940: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-21 20:45:14.706940: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-21 20:45:16.030705: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-21 20:45:16.030746: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: S03-SIAI
2022-05-21 20:45:16.030752: I tensorflow/stream_executor/cu

1/1 [==============================] - 1s 1s/step


100%|##########| 2/2 [00:00<00:00, 1222.47it/s]
2022-05-21 20:45:24.544375: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-21 20:45:24.609393: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-05-21 20:45:25.867780: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-21 20:45:25.867820: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: S03-SIAI
2022-05-21 20:45:25.867826: I tensorflow/stream_executor/

1/1 [==============================] - 1s 1s/step


2022-05-21 20:45:32.740712: W tensorflow/core/data/root_dataset.cc:247] Optimization loop failed: CANCELLED: Operation was cancelled


In [5]:
import numpy as np
import dgl
import torch

def mol_to_dgl(raw_graph, feature):
    # add edges
    #reactants_mol[mol, mol, mol], reactants_fea[(dfa, dfb), (dfa, dfb)]
    atoms = np.array([0] + [mol.GetNumAtoms() for mol in raw_graph])
    atom_cumsum = atoms.cumsum().tolist()[:-1]

    atom_fea = [df[0] for df in feature]
    bond_fea = [df[1] for df in feature]

    edges_src = [df.BeginAtomIdx.values.astype(int) for df in bond_fea]
    edges_dst = [df.EndAtomIdx.values.astype(int) for df in bond_fea]
    
    edges_src = [edges_src[i] + atom_cumsum[i] for i in range(len(atom_cumsum))]
    edges_dst= [edges_dst[i] + atom_cumsum[i] for i in range(len(atom_cumsum))]

    edges_src = np.concatenate(edges_src)
    edges_dst = np.concatenate(edges_dst)

    all_atom_fea = np.array(pd.concat(atom_fea))
    all_bond_fea = pd.concat(bond_fea)
    all_bond_fea = all_bond_fea.drop(['BeginAtomIdx', 'EndAtomIdx'], axis=1)
    all_bond_fea = np.array(all_bond_fea)
    
    graph = dgl.graph((edges_src, edges_src), num_nodes=all_atom_fea.shape[0])
    graph.ndata['feature'] = torch.from_numpy(all_atom_fea)
    graph.edata['feature'] = torch.from_numpy(all_bond_fea)
    # transform to bi-directed graph with self-loops
    graph = dgl.to_bidirected(graph, copy_ndata=True)
    graph = dgl.add_self_loop(graph)
    return graph

In [6]:
reactants_fea[0][0]

,PC,FN,FE,NSC,GC,MR,APol,SHI,TSEI,0,...,8,9,10,11,12,13,14,15,16,17
AtomIdx,,,,,,,,,,,,,,,,,,,,,
0,0.347135,1.00000,1.000000,1.000000,0.510199,0.636733,0.248233,0.000000,0.0,0,...,0,0,0,0,0,0,1,0,0,0
1,0.465139,0.10392,0.526888,0.445984,0.666147,0.099501,0.010932,0.129494,0.0,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
reactants_fea[0][1]

,SB,DB,TB,AB,CB,RB,SN,SA,SS,SR,BO,BL,BDE,BDFE,BeginAtomIdx,EndAtomIdx
BondIdx,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.192808,0.369449,0.351776,0.355857,1,0


In [8]:
g = mol_to_dgl(reactants_mol, reactants_fea)

In [ ]:
def get_atom_icnum(df):
    res = set()
    mol1 = Chem.MolFromSmiles(df['reactants'])
    mol2 = Chem.MolFromSmiles(df['products'])
    mol1 = Chem.AddHs(mol1)
    mol2 = Chem.AddHs(mol2)
    for atom in mol1.GetAtoms():
        anum = atom.GetAtomicNum()
        res.add(anum)
    for atom in mol2.GetAtoms():
        anum = atom.GetAtomicNum()
        res.add(anum)
    
    return res


In [ ]:
df['atom'] = df.apply(lambda x:get_atom_icnum(x), axis=1)

In [ ]:
df.head()

In [ ]:
all_atom = set()
for i in df['atom'].values:
    all_atom = all_atom | i

In [ ]:
len(all_atom)

In [ ]:
all_atom

In [ ]:
import numpy as np
c = np.array([0] + [2,3,4,6])
c.cumsum().tolist()[:-1]

In [ ]:
smiles

In [ ]:
dfa, dfb = get_rrd(smiles)